In [ ]:
#pykospacing downloading. 
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git


In [ ]:
#konlpy downloading. 
!set -x \
&& pip install konlpy \

#&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x


+ pip install konlpy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 8.5 MB/s 
     |████████████████████████████████| 453 kB 44.0 MB/s 


In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

Process is interrupted.


In [ ]:
#!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash


ERROR: Invalid requirement: '/tmp/mecab-python-0.996'
Hint: It looks like a path. File '/tmp/mecab-python-0.996' does not exist.


In [ ]:
!pip install git+https://github.com/kakaobrain/pororo.git


In [ ]:
from nltk.lm import NgramCounter
from nltk.util import ngrams
from pykospacing import Spacing
from google.colab import drive
drive.mount('/content/gdrive')
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
import json
import konlpy
from nltk.tokenize import word_tokenize
import re
from konlpy.tag import Okt
from konlpy.tag import Mecab, Komoran
import time
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from itertools import combinations
from itertools import product
import math
import pickle
"""
import sys
sys.setrecursionlimit(10**7)
"""

Mounted at /content/gdrive


'\nimport sys\nsys.setrecursionlimit(10**7)\n'

In [ ]:
#komoran=Komoran() mecab 만 사용할 거야. 이 칸은 텍스트 데이터 전처리에 관한 것.
#mecab=Mecab()
komoran=Komoran()
pattern_1=r"\([^)]*\)"
pattern_2=r"\.{2}"
#json file 을 더블리스트 형태로
def preprocess_sent(lst_text):
    wholelist=[]
    buf=str()
    subkor_1=re.sub(pattern=pattern_1, repl="", string=lst_text)
    subkor_2=re.sub(pattern=pattern_2, repl="", string=subkor_1)
    clear_list=re.compile("[가-힣]+|\.{1}").findall(subkor_2)
    for word in clear_list:
        if word==".":
            buf+=(word+" ")
            wholelist.append(buf)
            buf=str()
        
        else:
            buf+=(word+" ")
    return wholelist

#더블리스트 형태를 형태소 분석된 채의 더블리스트 형태로 
def sent_to_morph(lst_sent):
    double_list=[]
    for sentence in lst_sent:
        listed=komoran.morphs(sentence)
        #listed=mecab.morphs(sentence)
        double_list.append(listed)
    return double_list

In [ ]:
#윗 셀에 대한 예시. 파일 하나 가지고 놀기. 
with open("/content/gdrive/MyDrive/origin/BWSC217000049025.json", "r") as f:
    json_data=json.load(f)
#print(json.dumps(json_data))
kor_text=json_data["SJML"]["text"][1]["content"]
listed=preprocess_sent(kor_text)
double=sent_to_morph(listed)
print(double)


[['고객', '의', '경우', '올레', '를', '단기간', '이용', '하', 'ㄹ', '수', '있', '는', '올레', '플래시', '상품', '을', '포인트', '로', '결제', '하', 'ㄹ', '수', '있', '게', '되', 'ㄴ다', '.'], ['.'], ['.'], ['늘', 'ㄴ', '자사', '의', '프리미엄', '멤버', '십', '서비스', '이', 'ㄴ', '올레', '클럽', '회원', '을', '대상', '으로', '올레', '플래시', '상품', '을', '포인트', '이', 'ㄴ', '별로', '차감', '하', '는', '서비스', '를', '제공', '하', 'ㄴ다고', '일', '밝히', '었', '다', '.'], ['.'], ['.'], ['올레', '서비스', '는', '필요', '하', 'ㄹ', '때', '마다', '시간', '또는', '하루', '단위', '로', '구매', '하', '는', '서비스', '다', '.'], ['한', '시간', '은', '원', '하루', '는', '원', '이', '다', '.'], ['올레', '접속', '홈페이지', '에서', '결제', '후', '바로', '이용', '하', 'ㄹ', '수', '있', '다', '.'], ['또', '가로수길', '홍대', '삼청동', '등', '가맹점', '카페', '의', '매주', '수요일', '할인', '혜택', '을', '오', '는', '월', '말', '까지', '연장', '하', 'ㄴ다', '.'], ['하루', '에', '회', '이용한', '금액', '만큼', '별', '이', '차감', '되', 'ㄴ다', '.'], ['이용', '하', 'ㄹ', '수', '있', '는', '카페', '의', '위치', '등', '은', '올레', '클럽', '홈페이지', '에서', '확인', '하', 'ㄹ', '수', '있', '다', '.'], ['.'], ['.'], ['이', '외', '에', '도', '지나', 'ㄴ

In [ ]:
#학습된 모델을 바탕으로 단순히 숫자 세는 모델. 여기서는 ngramcounters 기준. 


def bigram_probs(model, pre_word, post_word, is_first=False):
  if is_first:
    unidict=dict(model.unigrams)
    uni_whole=np.sum(list(unidict.values()))
    uni_spec=unidict[pre_word]
    first_emer_prob=uni_spec/uni_whole
    word_dict=dict(model[[pre_word]])
    wholecount=model[pre_word]
    if wholecount!=0:

      try:
          post_count=word_dict[post_word]
      except:
          post_count=0
     

      emer_prob=(post_count/wholecount)*first_emer_prob

    else:
      emer_prob=0
  else:
    word_dict=dict(model[[pre_word]])
    wholecount=model[pre_word]
    if wholecount!=0:
      try:
        post_count=word_dict[post_word]
      except:
        post_count=0
      emer_prob=post_count/wholecount


  return emer_prob

def trigram_probs(model, pre_word, post_word):
    tulist=[]
    real_tulist=[]
    word_dict=dict(model[[pre_word]])
    
    for key in list(word_dict.keys()):
        if key !=pre_word:
            first_prob=bigram_probs(model, pre_word, key, is_first=True)
            second_prob=bigram_probs(model, key, post_word, is_first=False)
            whole_prob=first_prob*second_prob
            if whole_prob!=0:
                #normalization
                whole_prob=whole_prob*2
                tulist.append([pre_word, key, post_word, whole_prob])
                #이제부터는 greedy
                tu_df=pd.DataFrame(tulist, columns=["pre_word", "key", "post_word", "whole_prob"])
                tu_df=tu_df.sort_values(by=["whole_prob"], ascending=False)
                try:
                  tu_df=tu_df.iloc[:3, :]
                  real_tulist=tu_df.values.tolist()
                except:
                  real_tulist=tu_df=tu_df.values.tolist()
        else:
            continue
    return real_tulist

In [ ]:
#위에서 숫자 센 걸 바탕으로 
def canconnect(model, pre_word, post_word):
    possiblelist=[]
    biprob=bigram_probs(model, pre_word, post_word, is_first=True)
    if biprob!=0:
        possiblelist.append([pre_word, post_word, biprob])
    
    tulist=trigram_probs(model, pre_word, post_word)
    for li in tulist:
        possiblelist.append(li)

    return possiblelist
#여기까지는 그냥 코딩이고 이제부터 살짝 greedy 하게 진행
"""
    try:
      repre_prob=possiblelist[0][2]
      if len(possiblelist)>100:
        for i in range(100):
          repre_prob=max(repre_prob, possiblelist[i][2])



      
    except:
      print('connection failed!')

"""

    
#  가능한 케이스들과 이에 대한 확률 제시. 인덱스도 만들어야 해(가능한 경우의 수 개수)
def bigconnect(model, list_of_words):
    dummylist=[]
    index_list=[]
    for i in range(len(list_of_words)-1):
        connection=canconnect(model, list_of_words[i], list_of_words[i+1])
        if len(connection)!=0:
            n=len(connection)
            index_list.append(n)
            for possible in connection:
                dummylist.append(possible)
    
        else:
            print("connection failed")
            break
    return dummylist, index_list


In [ ]:
#위에서 주어진 dummylist 와 index_list를 바탕으로 깔끔한 출력을 위한 함수들.
def process(model, list_of_words):
    doublelists, index=bigconnect(model, list_of_words)
    for indexes in range(len(doublelists)):
        doublelists[indexes]=tuple(doublelists[indexes])
    return doublelists, index

def separate_connection(model,list_of_words):
    newconlist=[]
    pro_doublelists, pro_index=process(model, list_of_words)
    end, start=0, 0
    for i in pro_index:
        end+=i
        newconlist.append(pro_doublelists[start:end])

        start+=i
    return newconlist
  

#확률값이 너무 작아서 별로니 이를 예쁘게. 이 점수의 최댓값은 10000. 최저는 0. 
def pretty_prob(prob):
  prob=10000*prob
  prob=round(prob, 4)
  return prob  

def make_it_one(model, list_of_words):
    wholeconcat=[]
    wholeprob=1.0
    newlist=separate_connection(model, list_of_words)
    #separate connection을 거치면 
    maybe_one=list(product(*newlist))
    for i in maybe_one:
        for index in range(len(i)):
            wholeconcat.append(i[index][:-2])
            wholeprob*=i[index][-1]
        wholeconcat.append(tuple(list_of_words[-1]))
        wholeconcat.append(pretty_prob(wholeprob))
        wholeprob=1.0
    return wholeconcat



In [ ]:
#후처리 과정. 예쁘게 다듬어 주는 과정
def after_make(model, list_of_words):
  spacing=Spacing()
  text=make_it_one(model, list_of_words)
  new=[]
  sentence=""
  for t in text:
    if type(t)==tuple:
      for c in t:
        sentence+=c
    else:
      new.append([[spacing(sentence)], t])
      sentence=""

  return new



In [ ]:
def ngrams_mine(model, double_list):
  text_unigrams = [ngrams(sent, 1) for sent in double_list]
  text_bigrams  = [ngrams(sent, 2) for sent in double_list]
  model.update(text_unigrams+text_bigrams)
  return model


In [ ]:
n_model=NgramCounter()


#지금부터는 예시
text=[["a", "b", "c", "d"], ["d", "w", "c", "a"], ["c", "b", "d", "a"], ["k", "a", "t", "b", "c"], ["b", "a"], ["a", "m", "b"], ["a", "n","b"], ["a", "m", "b"], ["a", "x","b"], ["a", "t", "b"], ["a", "k", "b"], ["a", "m", "b"], ["a", "v", "b"]]
#ngrams_mine(n_model, text)
text2=[["a", "b", "c", "D"], ["d", "K", "c", "a"], ["c", "b", "c", "a"], ["a", "k", "b", "b"] ]
test_gram=ngrams_mine(n_model, text2)

#make_it_one(test_gram, ["a", "b"])


In [ ]:
#test_gram.unigrams

test_gram[["K"]]

FreqDist({'c': 1})

In [ ]:
#학습 phase. 일단 그냥 전체 데이터에 대한 리스트질을 해보자
#파일 받아서 전체과정
def file_to_string(filenick):
    dummylist=[]

    with open(filenick) as f:
        json_data=json.load(f)
    for i in range(0, 1000):
        try:
        
            kor_text=json_data["SJML"]["text"][i]["content"]
            dummylist.append(kor_text)
        except:
            break
    return dummylist

def pre_pipeline(stringed_text):
    listed=preprocess_sent(stringed_text)
    double=sent_to_morph(listed)
    
    return double

def whole_pipeline(filenick, given_model):
    dummies=file_to_string(filenick)
    
    for dummy in dummies:
        
        double_list=pre_pipeline(dummy)
        ngrams_mine(given_model, double_list)

In [ ]:
#실제 학습 phase
"""
d_path="미정"
filelist=os.listdir(d_path)
for filename in filelist:
    uniquefile=os.path.join(d_path, filename )
"""

[('a',), ('b',), 3333.3333]

In [ ]:
#생성된 객체 저장하는 시스템 만들기 (pickle 활용)
"""
with open("/content/gdrive/MyDrive/origin/testjames.p", "wb") as file:
  pickle.dump(n_model, file)
"""

In [ ]:
#생성된 객체 불러오는 시스템 만들기 (pickle 활용)
"""
with open("/content/gdrive/MyDrive/origin/testjames.p", "rb") as file:
  maybe=pickle.load(file)

maybe.N()
"""

'\nwith open("/content/gdrive/MyDrive/origin/testjames.p", "rb") as file:\n  maybe=pickle.load(file)\n\nmaybe.N()\n'

In [ ]:
#canconnect(n_model, "a", "b")
#trigram_probs(n_model, "a", "b")
#canconnect(n_model, "a", "b")

[['a', 'm', 'b', 0.4444444444444444],
 ['a', 't', 'b', 0.2222222222222222],
 ['a', 'k', 'b', 0.14814814814814814],
 ['a', 'n', 'b', 0.1111111111111111],
 ['a', 'x', 'b', 0.1111111111111111],
 ['a', 'v', 'b', 0.1111111111111111]]

In [ ]:
""" 
대충 학습
nn_model=NgramCounter()
d_path="/content/gdrive/MyDrive/origin"
filelist=os.listdir(d_path)
for filename in tqdm(filelist):
    uniquefile=os.path.join(d_path, filename )
    whole_pipeline(uniquefile, nn_model)
"""


' \n대충 학습\nnn_model=NgramCounter()\nd_path="/content/gdrive/MyDrive/origin"\nfilelist=os.listdir(d_path)\nfor filename in tqdm(filelist):\n    uniquefile=os.path.join(d_path, filename )\n    whole_pipeline(uniquefile, nn_model)\n'

In [ ]:
#취미 학습해보기
"""
n_model_hobby=NgramCounter()
d_path="/content/gdrive/MyDrive/TS1/취미"
filelist=os.listdir(d_path)
for filename in tqdm(filelist[:2000]):
    uniquefile=os.path.join(d_path, filename )
    whole_pipeline(uniquefile, n_model_hobby)
"""

'\nn_model_hobby=NgramCounter()\nd_path="/content/gdrive/MyDrive/TS1/취미"\nfilelist=os.listdir(d_path)\nfor filename in tqdm(filelist[:2000]):\n    uniquefile=os.path.join(d_path, filename )\n    whole_pipeline(uniquefile, n_model_hobby)\n'

In [ ]:
"""
with open("/content/gdrive/MyDrive/origin/test_hobby.p", "wb") as file:
  pickle.dump(n_model_hobby, file)
  """

In [ ]:
with open("/content/gdrive/MyDrive/origin/test_hobby.p", "rb") as file:
  hobby_komoran=pickle.load(file)

d_path="/content/gdrive/MyDrive/TS1/IT_과학"
filelist=os.listdir(d_path)

for filename in tqdm(filelist):
    uniquefile=os.path.join(d_path, filename )
    whole_pipeline(uniquefile, hobby_komoran)
    

with open("/content/gdrive/MyDrive/origin/test_hobbyit.p", "wb") as file:
  pickle.dump(hobby_komoran, file)

100%|██████████| 1217/1217 [1:07:31<00:00,  3.33s/it]


In [ ]:
"""
with open("/content/gdrive/MyDrive/origin/test_hobby.p", "rb") as file:
  hobby_komoran=pickle.load(file)
  """

'\nwith open("/content/gdrive/MyDrive/origin/test_hobby.p", "rb") as file:\n  hobby_komoran=pickle.load(file)\n  '

In [ ]:
#hobby_komoran.N() : 227859362
#canconnect(hobby_komoran, "한국", "자유")

227859362

In [ ]:
#열어서 시험해보기
with open("/content/gdrive/MyDrive/origin/test_hobbyit.p", "rb") as file:
  hobby_it=pickle.load(file)
  

In [ ]:
#hobby_it.N(): 359473776
c=after_make(hobby_it, ["강의",  "네트워크", "연결", "예쁘"])


[[['강의를 연결'], 0.0001], [['강의와 연결'], 0.0], [['강의로 연결'], 0.0]]

In [ ]:
bigconnect(hobby_it, ["강의", "네트워크", "연결", "예쁘"])

([['강의', '와', '네트워크', 9.377592719314168e-10],
  ['강의', '는', '네트워크', 7.667173348073857e-10],
  ['강의', '및', '네트워크', 6.247199937006409e-10],
  ['네트워크', '연결', 9.827055399063943e-07],
  ['네트워크', '를', '연결', 3.4455244874378286e-08],
  ['네트워크', '와', '연결', 2.0313505216732815e-08],
  ['네트워크', '로', '연결', 1.1978800906136205e-08],
  ['연결', '을', '예쁘', 9.212203683884024e-11],
  ['연결', '없이', '예쁘', 8.132823140854006e-11],
  ['연결', '이', '예쁘', 5.243953859219504e-11]],
 [3, 4, 3])

In [ ]:
dict_array=np.array(dict(hobby_it.unigrams).values())
#dict_array[dict_array.values()==1]

In [ ]:
#남아 있는 문제
#mecab
#검색 시간
#확률 너무 작은 거 해결


int

In [ ]:
a=pd.DataFrame(list(dict_array.reshape(-1))).T
a

,0
0,18
1,16
2,8
3,4
4,1
...,...
193530,1
193531,1
193532,1
193533,1


In [ ]:
hobby_it.N()

359473776

In [ ]:
len(list(dict(hobby_it.unigrams).keys()))
#타입 [] 강의

193535

In [ ]:
len(list((dict(hobby_it[["타입"]])).keys()))


307

In [ ]:
trigram_probs(hobby_it, "강의", "네트워크")


[['강의', '와', '네트워크', 9.377592719314168e-10],
 ['강의', '는', '네트워크', 7.667173348073857e-10],
 ['강의', '및', '네트워크', 6.247199937006409e-10]]

In [ ]:
bigram_probs(hobby_it, "네트워크", "강의")


0.0